In [31]:
# pull combination of aacta/afi and wikipedia data to write to wikidata.
# goal is to write at least all films, even if some individual recipients are not represented.

In [32]:
from rapidfuzz import process, fuzz
from requests_html import HTMLSession, HTML
import json
import numpy
import pandas
import pathlib
import pydash
import re
import requests
import time
import tqdm
import uuid
from wikibaseintegrator import WikibaseIntegrator, wbi_login, datatypes
from wikibaseintegrator.models import Claims, Qualifiers, References, Reference
from wikibaseintegrator.wbi_config import config


def fetch_links():

    ''' Derive links for all feature film awards from 1976 onwards. '''

    links = list()

    r = [x for x in range(1958, 2011) if x not in [1961, 1974]]
    c = list()
    for x in [1969, 1979, 1989, 1999]:
        l = r.index(x)+1
        c.append(r[:l])
        r = r[l:]
    c.append(r)

    for a in c:
        for b in a:
            links.append(f'https://www.aacta.org/aacta-awards/winners-and-nominees/range/{min(a)}-{max(a)}/year/{b}/')

    for x in ['inaugural-aacta-awards']+['2nd', '3rd', '4th', '5th', '6th', '7th']+[str(x) for x in range(2018,2022)]:
        if 'ina' not in x:
            x = f'{x}-aacta-awards'
        links.append(f'https://www.aacta.org/aacta-awards/winners-and-nominees/{x}')

    dataframe = pandas.DataFrame(columns=['ceremony', 'page'])

    for l in links:
        time.sleep(4)
        session = HTMLSession()
        r = session.get(l)
        for x in r.html.find('.filter-list__button', first=False):
            dataframe.loc[len(dataframe)] = [(pathlib.Path(l).stem), (x.attrs['data-load-winners'])]

    dataframe['ceremony'] = dataframe['ceremony'].str.replace('1995', '1995-2')
    dataframe['ceremony'] = dataframe['ceremony'].str.replace('1996', '1996-2')
    dataframe['ceremony'] = dataframe['ceremony'].str.replace('2006', '2006-2')

    dataframe = dataframe.loc[dataframe.page.str.contains('film', na=False)]
    dataframe = dataframe.loc[~dataframe.page.str.contains('short', na=False)]
    dataframe = dataframe.loc[~dataframe.page.str.contains('non', na=False)]

    return dataframe

def parse_awards(pages):

    ''' Extract award information from each page and assemble. '''

    stream = pandas.DataFrame()
    for x in tqdm.tqdm(pages.to_dict('records')):
        
        if x['ceremony'][:2] in ['19', '20']:
            x['year'] = x['ceremony'][:4]
        elif x['ceremony'] == '1995':
            x['year'] = '1995-2'        
        elif x['ceremony'] == 'inaugural-aacta-awards':
            x['year'] = '2011'
        else:
            x['year'] = '201'+x['ceremony'][:1] 

        time.sleep(4)
        url = f"https://www.aacta.org/aacta-awards/awards-history/{x['ceremony']}/loadwinners/{x['page']}"
        r = requests.get(url)
        if r.status_code == 200:
            data = pydash.get(json.loads(r.content.decode('utf-8')), "winners")   
            data = data.split('awards-list__title heading heading--lv3')[1:]
            print
            for d in data:
                award_data = pandas.DataFrame(columns=['candidate'])        
                html = HTML(html=d)
                award_data['candidate'] = [x.text for x in (html.find('.nominees-list__title'))]
                award_data['info'] = [x.text for x in (html.find('.nominees-list__info'))]
                award_data['winner'] = ['winner' in str(x) for x in (html.find('.nominees-list__item'))]
                award_data['award_name'] = d[2:].split('</h3>')[0]
                award_data['year'] = x['year']
                award_data['page'] = x['page']
                award_data['url'] = url
                stream = pandas.concat([stream, award_data])
        else:
            raise Exception(url, 'not found.')

    return stream

aacta_data = pathlib.Path.cwd() / 'data' / 'aacta_data.parquet'

if not aacta_data.exists():
    pages = fetch_links()
    dataframe = parse_awards(pages)
    dataframe.to_parquet(aacta_data, index=False)
else:
    dataframe = pandas.read_parquet(aacta_data)

print(len(dataframe))
dataframe.head()

2570


,candidate,info,winner,award_name,year,page,url
0,The Devil's Playground,Fred Schepisi,True,Best Film sponsored by the Australian Film Com...,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...
1,Picnic at Hanging Rock,"Patricia Lovell, Hal McElroy, Jim McElroy",False,Best Film sponsored by the Australian Film Com...,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...
2,The Devil's Playground,Fred Schepisi,True,Best Direction sponsored by Village Theatres,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...
3,Pure Shit,Bert Deling,False,Best Direction sponsored by Village Theatres,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...
4,End Play,Tim Burstall,False,Best Direction sponsored by Village Theatres,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...


In [33]:
# generalise award names.

awards = dataframe.copy()
awards = awards[['award_name']].drop_duplicates()

awards.loc[awards.award_name.str.contains('Best Film'), 'award_generic'] = 'Best Film'
awards.loc[awards.award_name.str.contains('Best Direction|Best Director|Best Achievement in Direction'), 'award_generic'] = 'Best Director'

awards.loc[awards.award_name.str.contains('Actress in a Leading Role|Best Lead Actress|Best Actress|Best Performance by an Actress'), 'award_generic'] = 'Best Lead Actress'
awards.loc[awards.award_name.str.contains('Actor in a Leading Role|Best Lead Actor|Best Actor|Best Performance by an Actor'), 'award_generic'] = 'Best Lead Actor'

awards.loc[awards.award_name.str.contains('Supporting Actress|Actress in a Supporting'), 'award_generic'] = 'Best Supporting Actress'
awards.loc[awards.award_name.str.contains('Supporting Actor|Actor in a Supporting'), 'award_generic'] = 'Best Supporting Actor'

for x in ['Cinematography', 'Editing', 'Sound', 'Costume Design', 'Production Design']:
    awards.loc[awards.award_name.str.contains(f'Best {x}|Best Achievement in {x}'), 'award_generic'] = f'Best {x}'
    
awards.loc[awards.award_name.str.contains('Screenplay'), 'award_generic'] = 'Best Screenplay'
awards.loc[awards.award_name.str.contains('Original Screenplay'), 'award_generic'] = 'Best Original Screenplay'
awards.loc[awards.award_name.str.contains('Adapted Screenplay|Screenplay Adapted'), 'award_generic'] = 'Best Adapted Screenplay'
awards.loc[awards.award_name.str.contains('Best Original Music|Best Original Score'), 'award_generic'] = 'Best Original Music Score'

awards = awards.loc[~awards.award_name.isin(['Best Soundtrack'])]
awards = awards.loc[~awards.award_generic.isin([numpy.nan])]

dataframe = pandas.merge(dataframe, awards, on='award_name', how='inner')
mask = dataframe['award_generic'].str.contains('Actor|Actress') 
dataframe.loc[mask, ['candidate', 'info']] = (dataframe.loc[mask, ['info', 'candidate']].values)
dataframe['candidate'] = dataframe['candidate'].str.split('\n').str[0]
dataframe['info'] = dataframe['info'].str.split('\n').str[0]

dataframe['info'] = dataframe['info'].str.split(',')
dataframe = dataframe.explode('info')
dataframe['info'] = dataframe['info'].str.strip()

print(len(dataframe))
dataframe.head()

3480


,candidate,info,winner,award_name,year,page,url,award_generic
0,The Devil's Playground,Fred Schepisi,True,Best Film sponsored by the Australian Film Com...,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Best Film
1,Picnic at Hanging Rock,Patricia Lovell,False,Best Film sponsored by the Australian Film Com...,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Best Film
1,Picnic at Hanging Rock,Hal McElroy,False,Best Film sponsored by the Australian Film Com...,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Best Film
1,Picnic at Hanging Rock,Jim McElroy,False,Best Film sponsored by the Australian Film Com...,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Best Film
2,The Devil's Playground,Fred Schepisi,True,Best Direction sponsored by Village Theatres,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Best Director


In [34]:
# better idea, deploy a "link searcher", target page where award should be located and locate link for element.

def link_searcher(row, col, delim):

    ''' Search relevant wiki page for link. '''

    source = pathlib.Path.cwd() / 'data' / f"{row['award_generic'].lower().replace(' ', '_')}.html"
    with open(source) as wikipedia:
        wikipedia = wikipedia.read()
        wikipedia = wikipedia.replace('&amp;', '&')
        wikipedia = wikipedia.replace('&#39;', "'")

        # &#39;

    for a in wikipedia.split('<tbody>')[1:]:
        for b in a.split('</tbody>')[0].split('</tr>'):
            for c in [x for x in b.split('</td>') if len(x)]:
                for d in c.split(delim):
                    if row[col].upper() in d.upper():
                        # print(d)
                        links = HTML(html=d)
                        if len(links.links) == 1:
                            result = list(links.links)[0]
                            if 'index.php' not in result:
                                return result

# download wikipedia pages to process from disk.

wikipedia_page = {'Best Film' : 'AACTA_Award_for_Best_Film',
    'Best Director' : 'AACTA_Award_for_Best_Direction',
    'Best Lead Actor': 'AACTA_Award_for_Best_Actor_in_a_Leading_Role',
    'Best Lead Actress': 'AACTA_Award_for_Best_Actress_in_a_Leading_Role', 
    'Best Supporting Actor': 'AACTA_Award_for_Best_Actor_in_a_Supporting_Role', 
    'Best Supporting Actress' : 'AACTA_Award_for_Best_Actress_in_a_Supporting_Role',
    'Best Cinematography' : 'AACTA_Award_for_Best_Cinematography', 
    'Best Editing' : 'AACTA_Award_for_Best_Editing', 
    'Best Sound': 'AACTA_Award_for_Best_Sound', 
    'Best Original Music Score': 'AACTA_Award_for_Best_Original_Music_Score',  
    'Best Costume Design': 'AACTA_Award_for_Best_Costume_Design',
    'Best Original Screenplay': 'AACTA_Award_for_Best_Original_Screenplay',
    'Best Production Design': 'AACTA_Award_for_Best_Production_Design',
    'Best Adapted Screenplay': 'AACTA_Award_for_Best_Adapted_Screenplay',
    'Best Screenplay': 'Australian_Film_Institute_Award_for_Best_Screenplay'
    }

for k,v in wikipedia_page.items():
    save_file = pathlib.Path.cwd() / 'data' / f"{k.lower().replace(' ','_')}.html"
    if not save_file.exists():
        page = requests.get(f'https://en.wikipedia.org/wiki/{v}')
        with open(save_file, 'w') as save_page:
            save_page.write(page.text)

# various fixes to encourage matches.

dataframe['candidate'] = dataframe['candidate'].str.replace('Nostrodamus', 'Nostradamus')
dataframe['candidate'] = dataframe['candidate'].str.replace('B.M.X. Bandits', 'BMX Bandits', regex=False)
dataframe['candidate'] = dataframe['candidate'].str.replace('Tresspassers', 'Trespassers')
dataframe['candidate'] = dataframe['candidate'].str.replace('A Wreck, A Tangle', 'A Wreck A Tangle')
dataframe.loc[dataframe.candidate.str.contains('Civil Dead', na=False), 'candidate'] = 'Civil Dead'
dataframe.loc[dataframe.candidate.str.contains('Blueburger', na=False), 'candidate'] = 'Esther Blueburger'
dataframe.loc[dataframe.candidate.str.contains('Judy & Punch', na=False), 'candidate'] = 'Judy and Punch'
dataframe.loc[dataframe.candidate.str.contains('Hounds Of Love', na=False), 'candidate'] = 'Hounds Of Love'
dataframe.loc[dataframe.candidate.str.contains('Berlin Syndrome', na=False), 'candidate'] = 'Berlin Syndrome'
dataframe.loc[dataframe.candidate.str.contains('Grendel', na=False), 'candidate'] = 'Grendel'
dataframe.loc[dataframe.candidate.str.contains('Mr Reliable', na=False), 'candidate'] = 'Mr. Reliable'
dataframe.loc[dataframe.candidate.str.contains('Mullaway', na=False), 'candidate'] = 'Mull'
dataframe.loc[dataframe.candidate.str.contains('Beattie', na=False), 'candidate'] = 'Beatie'
dataframe.loc[dataframe.candidate.isin(['Lion (See-Saw Films)']), 'candidate'] = 'Lion'
dataframe.loc[dataframe.candidate.isin(['Me, Myself, I']), 'candidate'] = 'Me Myself I'
dataframe.loc[dataframe.candidate.isin(['Peter Rabbit™']), 'candidate'] = 'Peter Rabbit'
dataframe.loc[dataframe.candidate.isin(['Rabbit Proof Fence']), 'candidate'] = 'Rabbit-Proof Fence'
dataframe.loc[dataframe.candidate.isin(['Samson & Delilah']), 'candidate'] = 'Samson and Delilah'
dataframe.loc[dataframe.candidate.isin(['Love & Other Catastrophes']), 'candidate'] = 'Love and Other Catastrophes'
dataframe.loc[dataframe.candidate.isin(["Ali's Wedding (Matchbox Pictures)"]), 'candidate'] = "Ali's Wedding"
dataframe.loc[dataframe.candidate.isin(["Children Of The Silk Road"]), 'candidate'] = "The Children of Huang Shi"
dataframe.loc[dataframe.candidate.isin(["Sher Mountain Killing Mystery"]), 'candidate'] = "Sher Mountain Killings Mystery"
dataframe.loc[dataframe.candidate.isin(["Slate, Wyn And Me"]), 'candidate'] = "Slate, Wyn & Me"
dataframe.loc[dataframe.candidate.isin(["Don't Tell (Tojohage Productions)"]), 'candidate'] = "Don't Tell"
dataframe.loc[dataframe.candidate.isin(["The Butterfly Tree (Midwinter Films)"]), 'candidate'] = "The Butterfly Tree"
dataframe.loc[dataframe.candidate.isin(["Doing Time For Patsy Kline"]), 'candidate'] = "Doing Time For Patsy Cline"
dataframe.loc[dataframe.candidate.isin(["Ellie and Abbie (& Ellie's Dead Aunt)"]), 'candidate'] = "Ellie & Abbie (& Ellie's Dead Aunt)"
dataframe.loc[dataframe.candidate.isin(["Fresh Air"]), 'candidate'] = "In A Savage Land"
dataframe.loc[dataframe.candidate.isin(["Jasper Jones (Bunya Productions/Porchlight Films)"]), 'candidate'] = "Jasper Jones"
dataframe.loc[dataframe.candidate.isin(["The Right Hand Man"]), 'candidate'] = "The Right-Hand Man"
dataframe.loc[dataframe.candidate.isin(["The LEGO® Batman Movie"]), 'candidate'] = "The LEGO Batman Movie"
dataframe.loc[dataframe.candidate.isin(["This Won't Hurt A Bit!"]), 'candidate'] = "This Won't Hurt A Bit"
dataframe.loc[dataframe.candidate.isin(["To Have And To Hold"]), 'candidate'] = "To Have & to Hold"
dataframe.loc[dataframe.candidate.isin(["To Market, To Market"]), 'candidate'] = "To Market To Market"
dataframe.loc[dataframe.candidate.isin(["Tomorrow When The War Began"]), 'candidate'] = "Tomorrow, When The War Began"
dataframe.loc[dataframe.candidate.isin(["Weekend With Katie"]), 'candidate'] = "Weekend With Kate"
dataframe.loc[dataframe.candidate.isin(["Where The Outback Ends"]), 'candidate'] = "Daisy and Simon"
dataframe.loc[dataframe.candidate.isin(["The Sounds Of One Hand Clapping"]), 'candidate'] = "The Sound Of One Hand Clapping"
dataframe.loc[dataframe.candidate.isin(["The Night, The Prowler"]), 'candidate'] = "The Night The Prowler"
dataframe.loc[dataframe.candidate.isin(["The King Is Dead!"]), 'candidate'] = "The King Is Dead"
dataframe.loc[dataframe.candidate.isin(["The Getting Of Wisom"]), 'candidate'] = "The Getting Of Wisdom"
dataframe.loc[dataframe.candidate.isin(["The Big Steel"]), 'candidate'] = "The Big Steal"
dataframe.loc[dataframe.candidate.isin(["Streethero"]), 'candidate'] = "Street Hero"
dataframe.loc[dataframe.candidate.isin(["Shadows Of The Peacock"]), 'candidate'] = "Echoes of Paradise"
dataframe.loc[dataframe.candidate.isin(["My Forgotten Man"]), 'candidate'] = "Flynn"
dataframe.loc[dataframe.candidate.isin(["Lillian's Story"]), 'candidate'] = "Lilian's Story"

# these are films that are totally absent, and should really be added to wiki.
dataframe = dataframe.loc[~dataframe.candidate.isin([
    'Ascendant', 
    'June Again', 
    'Measure for Measure', 
    'Mushrooms', 
    'Razzle Dazzle', 
    'An Indecent Obsession', 
    'Compo', 
    'Envy', 
    'Fell', 
    'In A Savage Land',
    'Friends and Strangers', 
    'Undertow', 
    'West Of Sunshine', 
    'The Flood', 
    'Sanctuary', 
    'Pulse', 
    'Love In Ambush'])]

dataframe['candidate_wikipedia'] = dataframe.apply(link_searcher, col='candidate', delim='^', axis=1)
dataframe['info_wikipedia'] = dataframe.apply(link_searcher, col='info', delim=',', axis=1)

if len(dataframe.loc[dataframe.candidate_wikipedia.isin([None])]):
    raise Exception('film link not found.')
    
# dataframe = dataframe.sort_values(by='candidate')

print(len(dataframe))
dataframe.head(40)

3425


,candidate,info,winner,award_name,year,page,url,award_generic,candidate_wikipedia,info_wikipedia
0,The Devil's Playground,Fred Schepisi,True,Best Film sponsored by the Australian Film Com...,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Best Film,/wiki/The_Devil%27s_Playground_(1976_film),/wiki/Fred_Schepisi
1,Picnic at Hanging Rock,Patricia Lovell,False,Best Film sponsored by the Australian Film Com...,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Best Film,/wiki/Picnic_at_Hanging_Rock_(film),/wiki/Patricia_Lovell
1,Picnic at Hanging Rock,Hal McElroy,False,Best Film sponsored by the Australian Film Com...,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Best Film,/wiki/Picnic_at_Hanging_Rock_(film),/wiki/Hal_and_Jim_McElroy
1,Picnic at Hanging Rock,Jim McElroy,False,Best Film sponsored by the Australian Film Com...,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Best Film,/wiki/Picnic_at_Hanging_Rock_(film),/wiki/Hal_and_Jim_McElroy
2,The Devil's Playground,Fred Schepisi,True,Best Direction sponsored by Village Theatres,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Best Director,/wiki/The_Devil%27s_Playground_(1976_film),/wiki/Fred_Schepisi
3,Pure Shit,Bert Deling,False,Best Direction sponsored by Village Theatres,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Best Director,/wiki/Pure_Shit,/wiki/Bert_Deling
4,End Play,Tim Burstall,False,Best Direction sponsored by Village Theatres,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Best Director,/wiki/End_Play,/wiki/Tim_Burstall
5,Picnic at Hanging Rock,Peter Weir,False,Best Direction sponsored by Village Theatres,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Best Director,/wiki/Picnic_at_Hanging_Rock_(film),/wiki/Peter_Weir
6,The Devil's Playground,Simon Burke,True,Best Actor sponsored by Hoyts Theatres,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Best Lead Actor,/wiki/The_Devil%27s_Playground_(1976_film),/wiki/Simon_Burke
7,The Devil's Playground,Nick Tate,True,Best Actor sponsored by Hoyts Theatres,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Best Lead Actor,/wiki/The_Devil%27s_Playground_(1976_film),/wiki/Nick_Tate


In [35]:
# it is not desirable to run this each time, you need an updateable resource.
# dictionary for wikipedia -> wikidata is basically what we are thinking about.

# start with a list of wikipedia items

def convert_wikidata(wikipedia):

    ''' Pull wikidata id via wikipedia api. '''

    time.sleep(4)
    path = wikipedia.replace('/wiki/', '')
    path = f'https://en.wikipedia.org/w/api.php?format=json&action=query&prop=pageprops&ppprop=wikibase_item&redirects=1&titles={path}'
    r = requests.get(path).text
    if "wikibase_item" in r:
        wikidata = r.split('"wikibase_item":"')[1].split('"')[0]
        if len(wikidata):
            return wikidata

wikipedia_page = {'Best Film' : 'AACTA_Award_for_Best_Film',
    'Best Director' : 'AACTA_Award_for_Best_Direction',
    'Best Lead Actor': 'AACTA_Award_for_Best_Actor_in_a_Leading_Role',
    'Best Lead Actress': 'AACTA_Award_for_Best_Actress_in_a_Leading_Role', 
    'Best Supporting Actor': 'AACTA_Award_for_Best_Actor_in_a_Supporting_Role', 
    'Best Supporting Actress' : 'AACTA_Award_for_Best_Actress_in_a_Supporting_Role',
    'Best Cinematography' : 'AACTA_Award_for_Best_Cinematography', 
    'Best Editing' : 'AACTA_Award_for_Best_Editing', 
    'Best Sound': 'AACTA_Award_for_Best_Sound', 
    'Best Original Music Score': 'AACTA_Award_for_Best_Original_Music_Score',  
    'Best Costume Design': 'AACTA_Award_for_Best_Costume_Design',
    'Best Original Screenplay': 'AACTA_Award_for_Best_Original_Screenplay',
    'Best Production Design': 'AACTA_Award_for_Best_Production_Design',
    'Best Adapted Screenplay': 'AACTA_Award_for_Best_Adapted_Screenplay',
    'Best Screenplay': 'Australian_Film_Institute_Award_for_Best_Screenplay'
    }

dataframe = dataframe.replace({'award_generic': wikipedia_page})
wikipedia_items = list(dataframe.candidate_wikipedia.unique())+list(dataframe.info_wikipedia.unique())+list(dataframe.award_generic.unique())
wikipedia_items = [x for x in wikipedia_items if x != None]

conversion_file = pathlib.Path.cwd() / 'wikidata_conversion.json'

for x in tqdm.tqdm(wikipedia_items):
    with open(conversion_file) as conv_in:
        conv_in = json.load(conv_in)
    if x not in conv_in.keys():
        conv_in[x] = convert_wikidata(x)
        with open(conversion_file, 'w') as conv_out:
            json.dump(conv_in, conv_out)

with open(conversion_file) as conv_in:
    conv_in = json.load(conv_in)
    
dataframe = dataframe.replace({'candidate_wikipedia': conv_in})
dataframe = dataframe.replace({'info_wikipedia': conv_in})
dataframe = dataframe.replace({'award_generic': conv_in})

print(len(dataframe))
dataframe.head(40)

100%|██████████| 1358/1358 [00:00<00:00, 3417.69it/s]


3425


,candidate,info,winner,award_name,year,page,url,award_generic,candidate_wikipedia,info_wikipedia
0,The Devil's Playground,Fred Schepisi,True,Best Film sponsored by the Australian Film Com...,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Q3600406,Q3520601,Q703727
1,Picnic at Hanging Rock,Patricia Lovell,False,Best Film sponsored by the Australian Film Com...,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Q3600406,Q1421747,Q3369120
1,Picnic at Hanging Rock,Hal McElroy,False,Best Film sponsored by the Australian Film Com...,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Q3600406,Q1421747,Q5640961
1,Picnic at Hanging Rock,Jim McElroy,False,Best Film sponsored by the Australian Film Com...,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Q3600406,Q1421747,Q5640961
2,The Devil's Playground,Fred Schepisi,True,Best Direction sponsored by Village Theatres,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Q4649807,Q3520601,Q703727
3,Pure Shit,Bert Deling,False,Best Direction sponsored by Village Theatres,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Q4649807,Q7261128,Q22277550
4,End Play,Tim Burstall,False,Best Direction sponsored by Village Theatres,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Q4649807,Q5375802,Q7803256
5,Picnic at Hanging Rock,Peter Weir,False,Best Direction sponsored by Village Theatres,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Q4649807,Q1421747,Q55424
6,The Devil's Playground,Simon Burke,True,Best Actor sponsored by Hoyts Theatres,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Q4649797,Q3520601,Q7518377
7,The Devil's Playground,Nick Tate,True,Best Actor sponsored by Hoyts Theatres,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Q4649797,Q3520601,Q1423160


In [52]:
test = dataframe.copy()
print(test.award_generic.unique())

test.head()

['Q3600406' 'Q4649807' 'Q4649797' 'Q4649800' 'Q4649792' 'Q4649795'
 'Q4824156' 'Q4649799' 'Q4649808' 'Q4649826' 'Q4649817' 'Q4649801'
 'Q4649818' 'Q4649794' 'Q4649821']


,candidate,info,winner,award_name,year,page,url,award_generic,candidate_wikipedia,info_wikipedia
0,The Devil's Playground,Fred Schepisi,True,Best Film sponsored by the Australian Film Com...,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Q3600406,Q3520601,Q703727
1,Picnic at Hanging Rock,Patricia Lovell,False,Best Film sponsored by the Australian Film Com...,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Q3600406,Q1421747,Q3369120
1,Picnic at Hanging Rock,Hal McElroy,False,Best Film sponsored by the Australian Film Com...,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Q3600406,Q1421747,Q5640961
1,Picnic at Hanging Rock,Jim McElroy,False,Best Film sponsored by the Australian Film Com...,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Q3600406,Q1421747,Q5640961
2,The Devil's Playground,Fred Schepisi,True,Best Direction sponsored by Village Theatres,1976,feature-film,https://www.aacta.org/aacta-awards/awards-hist...,Q4649807,Q3520601,Q703727


In [61]:
# build up statements to write to wikidata via wbintegrator, add checks - eg that films are films (date matching), humans have roles as expected?
# minimum requirement is that all films have matches.

# write a test instance, nominated for best director 1986.

from wikibaseintegrator.wbi_enums import ActionIfExists

with open(pathlib.Path.home() / 'wikidata_login.json') as wd_cred:
    wd_cred = json.load(wd_cred)

config['USER_AGENT'] = 'aacta-wikidata (https://github.com/paulduchesne/aacta-wikidata)'
login_wikidata = wbi_login.Login(user=wd_cred['username'], password=wd_cred['password'], mediawiki_api_url='https://www.wikidata.org/w/api.php')

# okay you have to do slightly more work here, as the nominations individuals should be grouped as included info statements

author = dataframe.copy().head(10)
author = author[['year', 'url', 'award_generic', 'candidate_wikipedia', 'info_wikipedia']]
author = author.pivot_table(index=['year', 'url', 'award_generic', 'candidate_wikipedia'], aggfunc=lambda x: ','.join(x.astype(str).unique())).reset_index()

for x in tqdm.tqdm(author.to_dict('records')):

    time.sleep(4)

    if str(x['candidate_wikipedia'])[0] == 'Q' and str(x['award_generic'])[0] == 'Q':

        year = f"+{x['year']}-01-01T00:00:00Z"

        new_qualifiers = Qualifiers()
        new_qualifiers.add(datatypes.Time(prop_nr='P585', time=year, precision=9))

        nominees = x['info_wikipedia'].split(',')
        nominees = [n for n in nominees if n[0] == 'Q']

        if len(nominees):
            for n in nominees:
                new_qualifiers.add(datatypes.Item(prop_nr='P2453', value=str(n)))

        references = References()
        reference1 = Reference()
        reference1.add(datatypes.URL(prop_nr='P854', value=str(x['url'])))
        references.add(reference1)

        wbi = WikibaseIntegrator(login=login_wikidata)
        wd_item = wbi.item.get(str(x['candidate_wikipedia']), mediawiki_api_url='https://www.wikidata.org/w/api.php', login=login_wikidata)
        claim = datatypes.Item(prop_nr='P1411', value=str(x['award_generic']), qualifiers=new_qualifiers, references=references)    
        wd_item.claims.add(claim, action_if_exists=ActionIfExists.APPEND_OR_REPLACE)
        wd_item.write()

author.head()

MediaWiki login warnings messages:
* main: Subscribe to the mediawiki-api-announce mailing list at <https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/> for notice of API deprecations and breaking changes. Use [[Special:ApiFeatureUsage]] to see usage of deprecated features by your application.
* login: Main-account login via "action=login" is deprecated and may stop working without warning. To continue login with "action=login", see [[Special:BotPasswords]]. To safely continue using main-account login, see "action=clientlogin".
100%|██████████| 7/7 [00:37<00:00,  5.41s/it]


,year,url,award_generic,candidate_wikipedia,info_wikipedia
0,1976,https://www.aacta.org/aacta-awards/awards-hist...,Q3600406,Q1421747,"Q3369120,Q5640961"
1,1976,https://www.aacta.org/aacta-awards/awards-hist...,Q3600406,Q3520601,Q703727
2,1976,https://www.aacta.org/aacta-awards/awards-hist...,Q4649797,Q3520601,"Q7518377,Q1423160"
3,1976,https://www.aacta.org/aacta-awards/awards-hist...,Q4649807,Q1421747,Q55424
4,1976,https://www.aacta.org/aacta-awards/awards-hist...,Q4649807,Q3520601,Q703727
